In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
Store = 34
w = ['Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'] 
data = pd.read_csv('Store'+str(Store)+'\Store'+str(Store)+'_item_20180103-20190711.csv') 
data.head()

#Define Invalid Criteria
#Ticket Time absolute value: for example if TicketTime < 90 seconds, treat as invalid
MinTicketTime = 90
#Ticket Time in propportion to SentTime: for example if TicketTime < 0.3*SentTime, treat as invalid
MinRatio = 0.3
#Table Open Time Range: for example, treat Table Open Time between 30 and 240 minutes as valid
MinTime = 30
MaxTime  = 300

## Variable Type Cleaning, Filling Guest Count for OffSite Orders

In [3]:
#Specify Variable Format for Date-Time Variable
data.loc[:,'BusinessDate'] = pd.to_datetime(data.loc[:,'BusinessDate'])

date_time = ['OrderStartDateTime','NormalDateTime','BumpedDateTime','CookingDateTime']
for i in date_time:
    data.loc[:,i] = pd.to_datetime(data.loc[:,i], format="%Y-%m-%d %H:%M:%S.%f")

#Adjust Time Format
data['StartTime'] = data['StartTime'].astype('str')
data.loc[data['StartTime'].str.len()==3,'StartTime'] = '0'+data['StartTime']
data.loc[data['StartTime'].str.len()==2,'StartTime'] = '00'+data['StartTime']
data.loc[data['StartTime'].str.len()==1,'StartTime'] = '000'+data['StartTime']
data['StartTime'] = data['StartTime'].str.slice(0,2)+':'+data['StartTime'].str.slice(2,5)

#Add Hour, Half Hour, 15min Time Slot Identifier
data['HalfHourStart'] = np.where(data['StartTime'].str.slice(3,5)=='15',data['StartTime'].str.slice(0,3)+'00', data['StartTime'])
data['HalfHourStart'] = np.where(data['StartTime'].str.slice(3,5)=='45',data['StartTime'].str.slice(0,3)+'30', data['HalfHourStart'])

data['QuarterHour'] = np.where(data['StartTime'].str.slice(0,2)=='00', \
                               24,(data['StartTime'].str.slice(0,2)).astype('int')) + data['StartTime'].str.slice(3,5).astype('int')/60
data['QuarterHour'] = np.where(data['QuarterHour']<=4, data['QuarterHour']+24,data['QuarterHour'])

data['HalfHour'] = data['QuarterHour']*4 //2 /2
data['Hour'] = data['QuarterHour']*4//4

#Adjust Time Slot Format
data['DayHalfHour'] = data['BusinessDate'].dt.round('D') + pd.to_timedelta(data['HalfHourStart']+':00')
data['DayQuarterHour'] = data['BusinessDate'].dt.round('D') + pd.to_timedelta(data['StartTime']+':00')
data.loc[data['DayHalfHour'].dt.hour<=4,'DayHalfHour']=data['DayHalfHour']+pd.to_timedelta(1,'d')
data.loc[data['DayQuarterHour'].dt.hour<=4,'DayQuarterHour']=data['DayQuarterHour']+pd.to_timedelta(1,'d')

#Identify day of week: 0 represents Mon, 6 represents Sun 
data['DayOfWeek'] = data['BusinessDate'].dt.weekday
data['DayOfWeekName'] = data['BusinessDate'].dt.weekday_name
data['Week'] = data['BusinessDate'].dt.week
data['Weekday'] = np.where(data['DayOfWeek']<=4,1,0)
data['OrderTimeMin'] = data['OrderStartDateTime'].dt.round('T')

#Identify holiday
data['Holiday'] = np.where(data['NatHolidayDesc'].str.contains('/'),0,1)

#Code Guest for Off-Site as -1
data.loc[(data['ChannelName']=='Delivery')|(data['ChannelName']=='To Go Sales')|(data['ChannelName']=='Digital'),'GuestCount']=-1

### Exclude Out-of-Operating Hour Item

In [5]:
#Read operating hour from excel, with sheet name as Store number 
OperatingHour = pd.read_excel('Operating Hours.xlsx',sheet_name=str(Store))

In [6]:
OperatingHour

,Unnamed: 0,Open,Close
0,Monday,11,23.0
1,Tuesday,11,23.0
2,Wednesday,11,23.0
3,Thursday,11,23.0
4,Friday,11,24.5
5,Saturday,10,24.5
6,Sunday,10,23.0


In [7]:
#For everyday of the week, keep records if within operating hours. 
d=pd.DataFrame()
for i in w:
    d = d.append(data[((data['DayOfWeekName']==i)&(data['HalfHour']>=OperatingHour.loc[i,'Open'])&(data['HalfHour']<OperatingHour.loc[i,'Close']))])
data = d 
del d 

KeyError: 'Monday'

In [8]:
data['BusinessDate'].max()

Timestamp('2019-07-11 00:00:00')

In [9]:
data['BusinessDate'].min()

Timestamp('2018-01-03 00:00:00')

### Fill in Guest Count for Off-Premise Orders

In [10]:
#Count number of items in every check, update off-premise check's GuestCount with estimation (#items in check). 
GuestEst = data.groupby(['StoreKey','BusinessDate','CheckNum'])['ProductKey'].count().reset_index()
GuestEst.columns = ['StoreKey','BusinessDate','CheckNum','GuestEst']
data = data.merge(right=GuestEst, how = 'left',on=['StoreKey','BusinessDate','CheckNum'])
data.loc[data['GuestCount']<=0,'GuestCount'] = data.loc[pd.isnull(data['GuestCount']),'GuestEst']
data = data.drop(columns='GuestEst')
del GuestEst

In [11]:
data.columns

Index(['StoreKey', 'BusinessDate', 'DateKey', 'DayOfWeek', 'TimeKey',
       'FullHour', 'HalfHour', 'QuarterHour', 'StartTime', 'Holiday',
       'CheckNum', 'GuestCount', 'TableOpenMinutes', 'OpenHour', 'OpenMinute',
       'CloseHour', 'CloseMinute', 'ChannelKey', 'TypeofServiceNum',
       'ProductKey', 'CourseName', 'IXIName', 'MajorCodeName', 'MinorCodeName',
       'StationKey', 'StationName', 'SentTime', 'OrderStartDateTime',
       'NormalDateTime', 'CookingDateTime', 'BumpedDateTime', 'NatHolidayDesc',
       'EmployeeKey', 'TicketTime', 'RNK', 'PROD_RNK', 'ORDER_RNK',
       'TypeofServiceCat', 'ChannelName', 'StartTime.1', 'OffSiteOrder',
       'OnSiteOrder', 'TotalOrder', 'OnSiteItem', 'OffSiteItem', 'TotalItem',
       'HalfHourStart', 'Hour', 'DayHalfHour', 'DayQuarterHour',
       'DayOfWeekName', 'Week', 'Weekday', 'OrderTimeMin'],
      dtype='object')

### Remove Non-Frequent Item

In [12]:
#Remove Items sold less than 100 times
data = data[data['ProductKey'].isin(list(data['ProductKey'].value_counts()[data['ProductKey'].value_counts()>=100].index))]

# Fill in Invalid Ticket Time

In [13]:
#Define less than 100 seconds or less than 30% of SentTime as invalid TicketTime
data['TicketTime'] = np.where((data['TicketTime']<=MinTicketTime)|(data['TicketTime']/data['SentTime']<MinRatio), np.NaN, data['TicketTime'])

In [14]:
#Count of Total Missing+Invalid Ticket Time
pd.isnull(data['TicketTime']).sum()

19646

In [15]:
### Define function for data filling ###
#Paramaeter explanation: data type in parentheses. 
#dataset: (pandas dataframe)
#objective: name of the target column to be filled. (string)
#groupby: name of the columns used to group by and fill missing value. (list of strings)
#measure: the measure taken from each group. (string: 'mean', 'median', or other pandas default aggregation function)
#countcondition: criteria to use selected measure to fill in missing value. Measure will be adopted if calculated from number of valid records exceeding this criteria. 
#percentagecondition: criteria to use selected measure to fill in missing value. Measure will be adopted if valid records / total records exceeds this criteria. 
def fill_invalid(dataset, objective, groupby, measure, countcondition, percentagecondition): 
    group = dataset.groupby(groupby)[objective].agg({objective:[lambda x: x.shape[0],'count',measure]}).reset_index()
    group.columns = groupby+['Count','CountValid',measure]
    group = group[(group['CountValid']>=countcondition)&(group['CountValid']/group['Count'])>=percentagecondition]
    d = dataset.merge(right=group, how='left',on=groupby)
    d[objective] = np.where((pd.isnull(d[objective]))&(pd.isnull(d[measure])==False),d[measure],d[objective])
    return list(d[objective])
#returned object: list of values in the same order as input column, with missing value filled (if criterion met) and other values remain the same. 

### Fill in Invalid for Holidays

In [16]:
#Initial #Missing
pd.isnull(data[(data['Holiday']==1)]['TicketTime']).sum() 

752

In [17]:
#Group by criteria: Product Key & HalfHour, then Product Key & Hour. 
for i in ['HalfHour','Hour',None]: 
    g = ['ProductKey','Holiday'] 
    if i != None:
        g = g + [i]
    data.loc[data['Holiday']==1,'TicketTime'] = fill_invalid(data[data['Holiday']==1],'TicketTime',g,'median',25,0.5)
    #Pring the current group-by criteria and the result after filling (#missing)
    print(i,pd.isnull(data[(data['Holiday']==1)]['TicketTime']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


HalfHour 686
Hour 519
None 42


## Fill In Invalid For Non-Holidays

In [18]:
#Initial #Missing
pd.isnull(data[(data['Holiday']==0)]['TicketTime']).sum() 

18894

In [19]:
for i in ['DayOfWeek','Weekday']:
    for j in ['HalfHour','Hour']: 
        g = ['ProductKey','Holiday']+[i]+[j]
        data.loc[data['Holiday']==0,'TicketTime'] = fill_invalid(data[data['Holiday']==0],'TicketTime',g,'median',25,0.5)
        print(i,j,pd.isnull(data[(data['Holiday']==0)]['TicketTime']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


DayOfWeek HalfHour 8768
DayOfWeek Hour 4142
Weekday HalfHour 1946
Weekday Hour 958


## Fill in the rest of invalid Ticket Time

In [20]:
for i in ['DayOfWeek','Weekday']:
    for j in ['HalfHour','Hour']: 
        g = ['ProductKey']+[i]+[j]
        data['TicketTime'] = fill_invalid(data,'TicketTime',g,'median',25,0.5)
        print(i,j,pd.isnull(data['TicketTime']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


DayOfWeek HalfHour 989
DayOfWeek Hour 983
Weekday HalfHour 981
Weekday Hour 928


## Fill in All the rest with Sent Time

In [21]:
data['TicketTime'] = np.where(pd.isnull(data['TicketTime']),data['SentTime'],data['TicketTime'])

In [22]:
#Check remaining missing value
pd.isnull(data['TicketTime']).sum()

0

In [23]:
#Update BumpedDateTime from filled TicketTime
data['BumpedDateTime_fill'] = data['NormalDateTime']+pd.to_timedelta(data['TicketTime'],unit='s')

In [24]:
#Number of Items with updated BumpedDateTime
(data['BumpedDateTime_fill']!=data['BumpedDateTime']).sum()

19646

# Fill In Invalid Table Open Time

In [25]:
#Separting On/Off Premise Orders
data['TypeofServiceNum'] = np.where(data['ChannelName'].isin(['Dining Room','Bar','Bar Dining','Patio']),0,1)

In [26]:
# Define less than 30 minutes and more than 300 minutes as invalid
data['TableOpenMinutes'] = np.where((data['TypeofServiceNum']==0)&((data['TableOpenMinutes']<MinTime)|(data['TableOpenMinutes']>MaxTime)), np.NaN, data['TableOpenMinutes'])

In [27]:
pd.isnull(data[(data['TypeofServiceNum']==0)]['TableOpenMinutes']).sum() 

3743

### Fill in Holiday Table Open Time

In [28]:
pd.isnull(data[(data['TypeofServiceNum']==0)&(data['Holiday']==1)]['TableOpenMinutes']).sum() 

143

In [29]:
for i in ['HalfHour','Hour',None]: 
    g = ['GuestCount','Holiday'] 
    if i != None:
        g = g + [i]
    data.loc[(data['Holiday']==1)&(data['TypeofServiceNum']==0),'TableOpenMinutes'] = fill_invalid(data[(data['Holiday']==1)&(data['TypeofServiceNum']==0)],'TableOpenMinutes',g,'median',30,0.7)
    print(i,pd.isnull(data[(data['Holiday']==1)&(data['TypeofServiceNum']==0)]['TableOpenMinutes']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


HalfHour 2
Hour 1
None 0


### Fill in NonHoliday Table Open Time

In [30]:
for i in ['DayOfWeek','Weekday']:
    for j in ['HalfHour','Hour']: 
        g = ['GuestCount','Holiday'] +[i,j]
        data.loc[(data['Holiday']==0)&(data['TypeofServiceNum']==0),'TableOpenMinutes'] = fill_invalid(data[(data['Holiday']==0)&(data['TypeofServiceNum']==0)],'TableOpenMinutes',g,'median',30,0.7)
        print(i,j,pd.isnull(data[(data['Holiday']==0)&(data['TypeofServiceNum']==0)]['TableOpenMinutes']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


DayOfWeek HalfHour 104
DayOfWeek Hour 85
Weekday HalfHour 68
Weekday Hour 60


### Fill in the rest of Table Open Time

In [31]:
#Fill in Holiday and Non-Holiday based on Day and Time 
for i in ['DayOfWeek','Weekday']:
    for j in ['HalfHour','Hour']: 
        g = ['GuestCount'] +[i,j]
        data.loc[(data['TypeofServiceNum']==0),'TableOpenMinutes'] = fill_invalid(data[(data['TypeofServiceNum']==0)],'TableOpenMinutes',g,'median',25,0.5)
        print(i,j,pd.isnull(data[(data['TypeofServiceNum']==0)]['TableOpenMinutes']).sum())

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


DayOfWeek HalfHour 60
DayOfWeek Hour 60
Weekday HalfHour 59
Weekday Hour 59


In [32]:
#Try to Fill in All based on Party Size
data.loc[(data['TypeofServiceNum']==0),'TableOpenMinutes'] = fill_invalid(data[(data['TypeofServiceNum']==0)],'TableOpenMinutes',['GuestCount'],'median',20,0.5)

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


In [33]:
#Check if all values filled
pd.isnull(data[(data['TypeofServiceNum']==0)]['TableOpenMinutes']).sum()

27

In [34]:
#Change Filling Criterion to Fill All based on Party Size
data.loc[(data['TypeofServiceNum']==0),'TableOpenMinutes'] = fill_invalid(data[(data['TypeofServiceNum']==0)],'TableOpenMinutes',['GuestCount'],'median',5,0.5)

C:\Users\jrush\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


In [35]:
#Update Table Open and Close Time based on Table Open Duration. 
data['OpenDateTime'] = data['BusinessDate']+pd.to_timedelta(data['OpenHour'],unit='h')+pd.to_timedelta(data['OpenMinute'],unit='m')
data['CloseDateTime'] = data['OpenDateTime']+pd.to_timedelta(data['TableOpenMinutes'],unit='m')

# Count Concurrent with SQL

### Aggregate From Item-Level to Order & Table

In [36]:
#Order: each row is one order with start and bump time. Used for #Concurrent Order counts. 
#Notice: batches of order within the same check is considered separate orders. 
#This is the base of JOINs, since concurrent counts are calculated based on order start time and items within an order shares the same count
orders = data.sort_values(by='BumpedDateTime_fill',ascending=False).\
drop_duplicates(subset=['StoreKey','BusinessDate','CheckNum','OrderTimeMin'],keep='first')

In [37]:
#Table: each row is one on-premise check (table) with table open and close time. Used for #On-Premise Guest counts. 
guests = orders[orders['TypeofServiceNum']==0].sort_values(by='BumpedDateTime_fill', ascending=False).\
drop_duplicates(subset=['StoreKey','BusinessDate','CheckNum'],keep='first')[['StoreKey','BusinessDate','CheckNum','GuestCount','OpenHour','OpenMinute','TableOpenMinutes','BumpedDateTime_fill']]
guests['OpenDateTime'] = guests['BusinessDate']+pd.to_timedelta(guests['OpenHour'],unit='h')+pd.to_timedelta(guests['OpenMinute'],unit='m')
guests['CloseDateTime'] = guests['OpenDateTime']+pd.to_timedelta(guests['TableOpenMinutes'],unit='m')
guests['CloseDateTime'] = np.where(guests['CloseDateTime']<=guests['BumpedDateTime_fill'],guests['BumpedDateTime_fill']+pd.to_timedelta(10,unit='m'),guests['CloseDateTime'])

In [38]:
#Keep only columns used in SQL JOIN
orders = orders[['StoreKey','BusinessDate','OrderStartDateTime','BumpedDateTime_fill','OrderTimeMin','TypeofServiceNum','CheckNum','StationName']]
guests = guests[['StoreKey','BusinessDate','OpenDateTime','CloseDateTime','GuestCount']]

In [39]:
import sqlite3

In [40]:
%%time
#Create 'database' in memory
conn = sqlite3.connect(':memory:')
#Write talbes into the 'database'
data[['StoreKey','BusinessDate','OrderStartDateTime','BumpedDateTime_fill','OrderTimeMin','TypeofServiceNum','CheckNum','StationName']].to_sql('BASE2', conn, index=False)
orders.to_sql('ORDERS', conn, index=False)
guests.to_sql('GUESTS', conn, index=False)

Wall time: 21.1 s


In [41]:
#Join orders to the base: #Concurrent Orders
qry = '''
SELECT
ORDERS.BusinessDate, ORDERS.CheckNum, ORDERS.OrderTimeMin, ORDERC.TypeofServiceNum,
COUNT(ORDERC.CheckNum) 'ORDERCOUNT'
FROM ORDERS 
JOIN ORDERS ORDERC 
	ON ORDERC.BusinessDate = ORDERS.BusinessDate 
	AND ORDERC.StoreKey = ORDERS.StoreKey
	AND ORDERC.CheckNum <> ORDERS.CheckNum 
	AND ORDERC.BumpedDateTime_fill > ORDERS.OrderTimeMin
	AND ORDERC.OrderStartDateTime <= ORDERS.OrderTimeMin
GROUP BY ORDERS.BusinessDate, ORDERS.CheckNum, ORDERS.OrderTimeMin, ORDERC.TypeofServiceNum
'''

In [42]:
%%time
OrderCount = pd.read_sql_query(qry,conn)

Wall time: 2min 4s


In [43]:
#Specify Variable Type
OrderCount['BusinessDate'] = pd.to_datetime(OrderCount['BusinessDate'])
OrderCount['OrderTimeMin'] = pd.to_datetime(OrderCount['OrderTimeMin'])
#Pivot the shape of OrderCount
#End result: OnSite and OffSite Orders in separate columns
OrderCount = OrderCount.groupby(['BusinessDate','CheckNum','OrderTimeMin','TypeofServiceNum'])['ORDERCOUNT'].mean().unstack().reset_index()
OrderCount.columns=['BusinessDate','CheckNum','OrderTimeMin','OnSiteOrder_fill','OffSiteOrder_fill'] 
#Merge the Concurrent Counts back to data
data = data.merge(right=OrderCount,how='left',on=['BusinessDate','CheckNum','OrderTimeMin'])

In [44]:
#NA occurs if no other orders in process at the time point, fill with 0
data['OnSiteOrder_fill']=data['OnSiteOrder_fill'].fillna(0)
data['OffSiteOrder_fill']=data['OffSiteOrder_fill'].fillna(0)

In [45]:
#Number of records updated: 
print('#OnSiteOrder Updated:',(data['OnSiteOrder_fill'] != data['OnSiteOrder']).sum())
print('#OffSiteOrder Updated:',(data['OffSiteOrder_fill'] != data['OffSiteOrder']).sum())

#OnSiteOrder Updated: 128207
#OffSiteOrder Updated: 12412


In [46]:
qry = '''
SELECT
ORDERS.OrderTimeMin, ORDERS.BusinessDate, ORDERS.CheckNum, PRODCOUNT.TypeofServiceNum,
COUNT(PRODCOUNT.CheckNum) 'PRODCOUNT'
FROM ORDERS 
JOIN BASE2 PRODCOUNT ON 
	PRODCOUNT.BusinessDate = ORDERS.BusinessDate 
	AND PRODCOUNT.StoreKey = ORDERS.StoreKey
	AND PRODCOUNT.CheckNum <> ORDERS.CheckNum 
	AND PRODCOUNT.BumpedDateTime_fill > ORDERS.OrderTimeMin 
	AND PRODCOUNT.OrderStartDateTime <= ORDERS.OrderTimeMin
GROUP BY ORDERS.OrderTimeMin, ORDERS.BusinessDate, ORDERS.CheckNum, PRODCOUNT.TypeofServiceNum
'''

In [47]:
%%time
ProdCount = pd.read_sql_query(qry,conn)
ProdCount['BusinessDate'] = pd.to_datetime(ProdCount['BusinessDate'])
ProdCount['OrderTimeMin'] = pd.to_datetime(ProdCount['OrderTimeMin']) 
ProdCount = ProdCount.groupby(['BusinessDate','CheckNum','OrderTimeMin','TypeofServiceNum'])['PRODCOUNT'].mean().unstack().reset_index()
ProdCount.columns=['BusinessDate','CheckNum','OrderTimeMin','OnSiteItem_fill','OffSiteItem_fill']


Wall time: 5min 1s


In [48]:
#Merge concurrent count back to data
data = data.merge(right = ProdCount,how='left',on=['BusinessDate','CheckNum','OrderTimeMin'])

In [49]:
#NA occurs if no other items in process at the time point, fill with 0
data['OnSiteItem_fill']=data['OnSiteItem_fill'].fillna(0)
data['OffSiteItem_fill']=data['OffSiteItem_fill'].fillna(0)

In [50]:
#Number of records updated: 
print('#OnSiteItem Updated:',(data['OnSiteItem_fill'] != data['OnSiteItem']).sum())
print('#OffSiteItem Updated:',(data['OffSiteItem_fill'] != data['OffSiteItem']).sum())

#OnSiteItem Updated: 315966
#OffSiteItem Updated: 30372


In [51]:
#Guest Count
qry = '''
SELECT
ORDERS.OrderTimeMin, ORDERS.BusinessDate, ORDERS.CheckNum,
SUM(GUESTCOUNT.GuestCount) 'OnSiteGuest'
FROM ORDERS 
JOIN GUESTS GUESTCOUNT ON 
	GUESTCOUNT.BusinessDate = ORDERS.BusinessDate 
	AND GUESTCOUNT.StoreKey = ORDERS.StoreKey
	AND GUESTCOUNT.CloseDateTime > ORDERS.OrderTimeMin
	AND GUESTCOUNT.OpenDateTime <= ORDERS.OrderTimeMin
GROUP BY ORDERS.BusinessDate, ORDERS.CheckNum, ORDERS.OrderTimeMin
'''

In [52]:
%%time
GuestCount = pd.read_sql_query(qry,conn)
GuestCount['BusinessDate'] = pd.to_datetime(GuestCount['BusinessDate'])
GuestCount['OrderTimeMin'] = pd.to_datetime(GuestCount['OrderTimeMin'])

Wall time: 2min 28s


In [53]:
#Merge concurrent count back to data
data = data.merge(right = GuestCount,how='left',on=['BusinessDate','CheckNum','OrderTimeMin']) 

In [54]:
data['OnSiteGuest'] = data['OnSiteGuest'].fillna(0) 

In [54]:
#ProdCount = ProdCount.groupby(['BusinessDate','CheckNum','OrderTimeMin'])['PRODCOUNT'].mean().unstack().reset_index()
#ProdCount.columns=['BusinessDate','CheckNum','OrderTimeMin','OnSiteItem_fill','OffSiteItem_fill']

In [55]:
#Update the original concurrent count before cleaning
data['BumpedDateTime'] = data['BumpedDateTime_fill']
data['OnSiteItem'] = data['OnSiteItem_fill']
data['OffSsiteItem'] = data['OffSiteItem_fill']
data['OnSiteOrder'] = data['OnSiteOrder_fill']
data['OffSiteOrder'] = data['OffSiteOrder_fill']

In [56]:
data.to_csv('Store'+str(Store)+'\Store'+str(Store)+'_item_filled_20180103-20190711.csv') 

# Concurrent Count during 15 min

In [57]:
#Construct the Time Frame Table
#Each row is a Day-15min period
time = data.groupby(['StoreKey','BusinessDate','TimeKey']).agg({'Holiday':'mean','DayQuarterHour':'first','QuarterHour':'mean','StartTime':'first'}).reset_index()
time['DayQuarterHourEnd'] = time['DayQuarterHour']+pd.to_timedelta(15,unit='m') 

In [58]:
%%time
#Add time frame table to database, as the base for concurrent counts
time.to_sql('DAYTIME',conn,index=False)

Wall time: 266 ms


In [59]:
qry = '''
SELECT
DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, ORDERC.TypeofServiceNum, COUNT(ORDERC.CheckNum) 'ORDERCOUNT'
FROM DAYTIME
JOIN ORDERS ORDERC 
	ON DAYTIME.BusinessDate = ORDERC.BusinessDate
	AND DAYTIME.StoreKey = DAYTIME.StoreKey
	AND ORDERC.OrderStartDateTime < DAYTIME.DayQuarterHourEnd AND ORDERC.BumpedDateTime_fill > DAYTIME.DayQuarterHour
GROUP BY DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, ORDERC.TypeofServiceNum
'''

In [60]:
%%time
OrderCount1 = pd.read_sql_query(qry,conn)
OrderCount1['BusinessDate'] = pd.to_datetime(OrderCount1['BusinessDate'])
OrderCount1['DayQuarterHour'] = pd.to_datetime(OrderCount1['DayQuarterHour']) 

Wall time: 9.75 s


In [61]:
OrderCount1 = OrderCount1.groupby(['BusinessDate','DayQuarterHour','TypeofServiceNum'])['ORDERCOUNT'].mean().unstack().reset_index()
OrderCount1.columns= ['BusinessDate','DayQuarterHour','OnSiteOrder','OffSiteOrder']

In [62]:
qry = '''
SELECT
DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, PROD.TypeofServiceNum, COUNT(PROD.CheckNum) 'PRODCOUNT'
FROM DAYTIME
JOIN BASE2 PROD ON DAYTIME.BusinessDate = PROD.BusinessDate
	AND PROD.OrderStartDateTime < DAYTIME.DayQuarterHourEnd AND PROD.BumpedDateTime_fill > DAYTIME.DayQuarterHour
GROUP BY DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, PROD.TypeofServiceNum
'''

In [63]:
%%time
ProdCount1 = pd.read_sql_query(qry,conn)
ProdCount1['BusinessDate'] = pd.to_datetime(ProdCount1['BusinessDate'])
ProdCount1['DayQuarterHour'] = pd.to_datetime(ProdCount1['DayQuarterHour']) 

Wall time: 41.5 s


In [64]:
ProdCount1 = ProdCount1.groupby(['BusinessDate','DayQuarterHour','TypeofServiceNum'])['PRODCOUNT'].mean().unstack().reset_index()
ProdCount1.columns= ['BusinessDate','DayQuarterHour','OnSiteItem','OffSiteItem'] 

In [65]:
qry = '''
SELECT
DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, PROD.StationName, COUNT(PROD.CheckNum) 'PRODCOUNT'
FROM DAYTIME
JOIN BASE2 PROD ON DAYTIME.BusinessDate = PROD.BusinessDate
	AND PROD.OrderStartDateTime < DAYTIME.DayQuarterHourEnd AND PROD.BumpedDateTime_fill > DAYTIME.DayQuarterHour
GROUP BY DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, PROD.StationName
'''

In [66]:
%%time
StationCount1 = pd.read_sql_query(qry,conn)

Wall time: 57 s


In [67]:
StationCount1['BusinessDate'] = pd.to_datetime(StationCount1['BusinessDate'])
StationCount1['DayQuarterHour'] = pd.to_datetime(StationCount1['DayQuarterHour']) 
StationCount1 = StationCount1.groupby(['BusinessDate','DayQuarterHour','StationName'])['PRODCOUNT'].mean().unstack().reset_index()

In [68]:
qry = '''
SELECT
DAYTIME.BusinessDate, DAYTIME.DayQuarterHour, SUM(GUESTCOUNT.GuestCount) 'OnSiteGuest'
FROM DAYTIME
JOIN GUESTS GUESTCOUNT ON DAYTIME.BusinessDate = GUESTCOUNT.BusinessDate
	AND GUESTCOUNT.OpenDateTime < DAYTIME.DayQuarterHourEnd AND GUESTCOUNT.CloseDateTime > DAYTIME.DayQuarterHour
GROUP BY DAYTIME.BusinessDate, DAYTIME.DayQuarterHour
'''

In [69]:
%%time
GuestCount1 = pd.read_sql_query(qry,conn)

Wall time: 12.5 s


In [70]:
GuestCount1['BusinessDate'] = pd.to_datetime(GuestCount1['BusinessDate'])
GuestCount1['DayQuarterHour'] = pd.to_datetime(GuestCount1['DayQuarterHour']) 

In [71]:
time = time.merge(right=ProdCount1, how='left',on=['BusinessDate','DayQuarterHour'])
time = time.merge(right=StationCount1, how='left',on=['BusinessDate','DayQuarterHour'])
time = time.merge(right=OrderCount1, how='left',on=['BusinessDate','DayQuarterHour'])
time = time.merge(right=GuestCount1, how='left',on=['BusinessDate','DayQuarterHour'])

In [72]:
time = time.fillna(0)

In [73]:
time.to_csv('Store'+str(Store)+'\Store'+str(Store)+'_day-15min.csv')

# Concurrent Count during 15min: Bottoleneck Item in Each Order.